# GENERATE FIGURE INPUTS

This notebook provides functions and scripts for generating input files for visualization with D3.

---
- [WORKSPACE VARIABLES](#workspace-variables)
- **[SITE ARCHITECTURE](#site-architecture)**
- **[ESTIMATED HEMODYNAMICS](#estimated-hemodynamics)**
- **[EXACT HEMODYNAMICS](#exact-hemodynamics)**
- **[VASCULAR DAMAGE](#vascular-damage)**
- **[VASCULAR FUNCTION](#vascular-function)**
---

Each of the simulation sets has a corresponding script of the same name, which contains a class (of the same name) with relevant condition variables and methods to iterate through these condition.
The `loop` method will be the most often, to iterate through all conditions, extract relevant information, and then compile the information into a single output file.
The `loop` method works with both individual files for each condition (`.csv` or `.json`) or a `.tar.xz` compressed archive of the files for each condition.
For a given figure, not all conditions may be used.

All generated figure inputs are provided in the `analysis` directory.
As much as possible, files are provided in compressed archive form, using the provided `compress.sh` script, taking three arguments: `[NAME OF SIMULATION SET] [EXTENSION NAME] [json OR csv]`.
The corresponding arguments for each of the following figures is provided in the description.

The `generate.py` file contains functions to generate figure inputs.

In [ ]:
from scripts.generate import *

<a class="anchor" id="workspace-variables"></a>

### WORKSPACE VARIABLES

Set up workspace variables for analyzing simulations.

- **`DATA_PATH`** is the path to data files (`.tar.xz` files of compressed simulation outputs)
- **`RESULTS_PATH`** is the path to result files (`.pkl` files generated by parsing)
- **`ANALYSIS_PATH`** is the path for analysis files (`.json` and `.csv` files, `.tar.xz` compressed archives)

In [ ]:
DATA_PATH = "/path/to/data/files/"
RESULTS_PATH = "/path/to/result/files/"
ANALYSIS_PATH = "/path/to/analysis/files/"

<a class="anchor" id="site-architecture"></a>

### SITE ARCHITECTURE

Generate `SITE_ARCHITECTURE` figure inputs.

In [ ]:
from scripts.SITE_ARCHITECTURE import SITE_ARCHITECTURE

<a class="anchor" id="estimated-hemodynamics"></a>

### ESTIMATED HEMODYNAMICS

Generate `ESTIMATED_HEMODYNAMICS` figure inputs.

In [ ]:
from scripts.ESTIMATED_HEMODYNAMICS import ESTIMATED_HEMODYNAMICS

<a class="anchor" id="exact-hemodynamics"></a>

### EXACT HEMODYNAMICS

Generate `EXACT_HEMODYNAMICS` figure inputs.

In [ ]:
from scripts.EXACT_HEMODYNAMICS import EXACT_HEMODYNAMICS

<a class="anchor" id="vascular-damage"></a>

### VASCULAR DAMAGE

Generate `VASCULAR_DAMAGE` figure inputs.

In [ ]:
from scripts.VASCULAR_DAMAGE import VASCULAR_DAMAGE

<a class="anchor" id="vascular-function"></a>

### VASCULAR FUNCTION

Generate `VASCULAR_FUNCTION` figure inputs.

In [ ]:
from scripts.VASCULAR_FUNCTION import VASCULAR_FUNCTION